In [ ]:
import pandas as pd
import json
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

Data cleaning improvement: token-ize ingredients


In [ ]:
def format_ingredient_data(df):
    mlb = MultiLabelBinarizer()
    X = pd.DataFrame(mlb.fit_transform(df["ingredients"]), columns=mlb.classes_)
    y = df["cuisine"]
    return X, y


with open("data/yumly_train.json", "r") as json_f:
    df = pd.DataFrame(json.load(json_f))
    X, y = format_ingredient_data(df)

In [ ]:
# Drop columns in X which have a sum across all rows less than 10
X = X.loc[:, (X.sum(axis=0) >= 10)]

In [ ]:
# Group X by cuisine and sum the value of each column
ingredient_counts = X.groupby(y).sum()
# Normalize the ingredient counts by dividing each value by the sum of the row
ingredient_props = ingredient_counts.div(ingredient_counts.sum(axis=1), axis=0)

def get_principle_components(X, explained_variance_perc = .95):
  X_T = X.T
  U, S, _ = np.linalg.svd(X_T)
  n_components = np.argmax(np.cumsum(S / S.sum()) >= explained_variance_perc)
  vectors = pd.DataFrame(index = X.index)
  for i in range(n_components):
    vectors[f'{i+1}'] = (U[:, i] @ X_T) / (S[i] ** .05)
  return vectors

principle_components = get_principle_components(ingredient_props)
plt.figure(figsize=(10, 8))
plt.scatter(principle_components['1'], principle_components['2'])

for i, txt in enumerate(principle_components.index):
  plt.annotate(txt, (principle_components['1'][i], principle_components['2'][i]))

plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.title('PCA Scatter Plot of Ingredients by Cuisine')
plt.show()

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_model = RandomForestClassifier(n_estimators=100, max_features=10, n_jobs=8, min_samples_split=50, oob_score=True)
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

In [ ]:
# Perform feature selection using the trained RandomForest model
feature_importances = rf_model.feature_importances_
important_features = pd.Series(feature_importances, index=X_train.columns).sort_values(ascending=False)

# Select features that capture the first 95% of explainability
cumulative_importances = important_features.cumsum()
features_95 = cumulative_importances[cumulative_importances <= 0.95].index

# Remove any columns in X which are not in features_95
X = X[features_95]

# Redo the train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# retrain with thinner dataset
rf_model = RandomForestClassifier(n_estimators=100, max_features=10, n_jobs=8, min_samples_split=50, oob_score=True)
rf_model.fit(X_train, y_train)

# Predict on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Let's optimize using oob_score


In [ ]:
with open("data/recipes_raw_nosource_epi.json", "r") as json_f:
    epicurious_df = pd.DataFrame(json.load(json_f)).T.reset_index(drop=True)

with open("data/recipes_raw_nosource_fn.json", "r") as json_f:
    fn_df = pd.DataFrame(json.load(json_f)).T.reset_index(drop=True)
    
with open("data/recipes_raw_nosource_ar.json", "r") as json_f:
    ar_df = pd.DataFrame(json.load(json_f)).T.reset_index(drop=True)

df = pd.concat([epicurious_df, fn_df, ar_df], ignore_index=True)
df = df[["title", "instructions", "ingredients"]]
df = df.dropna().reset_index(drop = True)
df["ingredients"] = df["ingredients"].apply(
    lambda x: " ".join(x) if isinstance(x, list) else x)

n = len(features_95)
m = len(df)
recipe_ingredients = np.zeros((m, n))
for i, feature in enumerate(features_95):
  recipe_ingredients[:, i] = df["ingredients"].apply(lambda x: 1 if feature in x else 0)

recipe_ingredients = pd.DataFrame(recipe_ingredients, columns = features_95)

In [ ]:
cuisine_probabilities = rf_model.predict_proba(recipe_ingredients)

for i, cuisine in enumerate(rf_model.classes_):
  df[f'prob_{cuisine}'] = cuisine_probabilities[:, i]

df['predicted_cuisine'] = rf_model.predict(recipe_ingredients)
df['predicted_cuisine_prob'] = np.max(cuisine_probabilities, axis=1)

In [ ]:
meal_plan = df.sample(n=1)

meal_plan_ingredients = recipe_ingredients.loc[meal_plan.index]

# TODO: Need to remove my existing meal plan recipes
distances = cdist(meal_plan_ingredients, recipe_ingredients, metric='euclidean')

# Find the closest recipes for each predicted cuisine
closest_indices = []
for cuisine in rf_model.classes_:
  cuisine_indices = df[df['predicted_cuisine'] == cuisine].index
  cuisine_distances = distances[0][cuisine_indices]
  closest_cuisine_indices = cuisine_indices[np.argsort(cuisine_distances)[:5]]  # Get the 5 closest recipes for each cuisine
  closest_indices.extend(closest_cuisine_indices)

# Get the closest recipes
closest_recipes = df.iloc[closest_indices]

print("Closest Recipes:")
print(closest_recipes[["title", "predicted_cuisine", "predicted_cuisine_prob"]])

Outputs needed:
  - Model
  - List of features it needs

Code I can write from that
  - Given a recipe from the other dataset, do feature of extraction
  - Predict cuisine
